<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [35]:
import pandas as pd
import psycopg2
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

In [36]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python 


In [37]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [38]:
query_3_1 = f'''
select
    count(id) as count
from public.vacancies
'''

In [39]:
df_3_1 = pd.read_sql_query(query_3_1, connection)
print('Количество вакансий в БД:', df_3_1.iloc [0]['count'] )

Количество вакансий в БД: 49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [40]:
query_3_2 = f'''
select
    count(id) as count
from public.employers
'''

In [41]:
df_3_2 = pd.read_sql_query(query_3_2, connection)
print('Количество работодателей в БД:', df_3_2.iloc [0]['count'] )

Количество работодателей в БД: 23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [42]:
query_3_3 = f'''
select
    count(id) as count
from public.areas
'''

In [43]:
df_3_3 = pd.read_sql_query(query_3_3, connection)
print('Количество регионов охваченных БД:', df_3_3.iloc [0]['count'] )

Количество регионов охваченных БД: 1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [44]:
query_3_4 = f'''
select
    count(id) as count
from public.industries
'''

In [45]:
df_3_4 = pd.read_sql_query(query_3_4, connection)
print('Количество сфер деятельности охваченных БД:', df_3_4.iloc [0]['count'] )

Количество сфер деятельности охваченных БД: 294


***

                                    ВЫВОД
В базе содержится 49197 вакансий, 23501 работодателей, из 1362 Регионов, и охватывают 294 сферы деятельности.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [46]:
query_4_1 = f'''
select
    e.name as "'Регион'",
    count(v.name) as "'Колличество'"
from public.vacancies v
join public.areas e on v.area_id = e.id
group by 1
order by 2 desc
limit 10
'''

In [47]:
df_4_1 = pd.read_sql_query(query_4_1, connection)
df_4_1

,'Регион','Колличество'
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
5,Екатеринбург,1698
6,Нижний Новгород,1670
7,Казань,1415
8,Краснодар,1301
9,Самара,1144


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [48]:
query_4_2 = f'''
select
    count(v.name) as cnt
from public.vacancies v
where salary_from is not null or salary_to is not null
'''


In [49]:
df_4_2 = pd.read_sql_query(query_4_2, connection)
print('Количество вакансий где указана минимальная \
или максимальная ожидаемая зарплата:', df_4_2.iloc [0]['cnt'] )

Количество вакансий где указана минимальная или максимальная ожидаемая зарплата: 24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [50]:
query_4_3 = f'''
select
    ROUND(avg(salary_from)) as "'нижняя граница'",
    ROUND(avg(salary_to)) as "'верхняя граница'"
from public.vacancies v
'''

In [51]:
df_4_3 = pd.read_sql_query(query_4_3, connection)
df_4_3

,'нижняя граница','верхняя граница'
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [52]:
query_4_4 = f'''
select
    schedule as "'тип рабочего графика'",
    employment as "'тип трудоустройства'",
    count(v.name) as "'количество вакансий'"    
from public.vacancies v
group by 1, 2
order by 3 desc
'''

In [53]:
df_4_4 = pd.read_sql_query(query_4_4, connection)
df_4_4

,'тип рабочего графика','тип трудоустройства','количество вакансий'
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [54]:
query_4_5 = f'''
select
    experience as "'Требуемый опыт работы'",
    count(v.name) as "'Колличество вакансий'"    
from public.vacancies v
group by 1
order by 2
'''

In [55]:
df_4_5 = pd.read_sql_query(query_4_5, connection)
df_4_5

,'Требуемый опыт работы','Колличество вакансий'
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

                                    ВЫВОД
В пятерку лидеров-городов по колличеству вакансий входят Москва, Санкт-Питербург, Минск, Новосибирск, Алмааты Это самые крупные региональные центры СНГ.

24073 вакансии имеют верхнюю или нижнюю границу зарплаты. это означает что в половине вакансий не указанна зарплата

Средняя нижняя граница равна 71065 руб, а средняя верхняя 110537 руб.

2 по популярности график работы и тип трудоустройства является удаленная работа- Полная занятость.

По колличеству вакансий от требуемого опыта вакансии распределены следующим образом 
Более 6 лет — Нет опыта — От 3 до 6 лет — От 1 года до 3 лет что показывает что большинство вакансий имеют требование опыта работы От 1 года до 3 лет


# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [56]:
query_5_1 = f'''
select
    e.name as "'Работодатель'",
    count(v.name) as "'количество вакансий'"    
from public.vacancies v
join public.employers e on v.employer_id = e.id
group by 1
order by 2 desc
LIMIT 5
'''

In [57]:
df_5_1 = pd.read_sql_query(query_5_1, connection)
display(df_5_1)
print('На 1 месте по количеству вакансий находится работодатель:', df_5_1.iloc [0][0],',' 'а на 5', df_5_1.iloc [4][0] )

,'Работодатель','количество вакансий'
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


На 1 месте по количеству вакансий находится работодатель: Яндекс ,а на 5 Газпром нефть


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [58]:
query_5_2 = f'''
select
    a.name as "'Регион'",
    count(e.id) as "'Колличество работодателей'",
    count(v.id) as "'Колличество вакансий'"    
from public.areas a
left join public.employers e on e.area = a.id
left join public.vacancies v on v.area_id = a.id
group by 1
HAVING count(v.id) = 0
order by 2 desc
limit 10
'''

In [59]:
df_5_2 = pd.read_sql_query(query_5_2, connection)
display(df_5_2)
print('Регион в котором больше всего работодателей но нет вакансий:', df_5_2.iloc [0][0])

,'Регион','Колличество работодателей','Колличество вакансий'
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Ростовская область,18,0
5,Беларусь,18,0
6,Азербайджан,17,0
7,Нижегородская область,16,0
8,Республика Татарстан,16,0
9,Узбекистан,15,0


Регион в котором больше всего работодателей но нет вакансий: Россия


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [60]:
query_5_3 = f'''
select
    e.name as "'Работодатель'",
    count(DISTINCT v.area_id) as "'Колличество регионов'"    
from public.employers e
join public.vacancies v on e.id = v.employer_id
group by 1
order by 2 desc
limit 5
'''

In [61]:
df_5_3 = pd.read_sql_query(query_5_3, connection)
df_5_3

,'Работодатель','Колличество регионов'
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [62]:
query_5_4 = f'''
select
    count(e.id) as "'Колличество работодателей'"
from public.employers e
LEFT JOIN public.employers_industries ei ON e.id = ei.employer_id
WHERE ei.industry_id IS NULL
'''

In [63]:
df_5_4 = pd.read_sql_query(query_5_4, connection)
#df_5_4
print('Количество работодателей у которые отсутствует сфера деятельности:', df_5_4.iloc [0][0])

Количество работодателей у которые отсутствует сфера деятельности: 8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [64]:
query_5_5 = f'''
SELECT e.name
FROM public.employers e
LEFT JOIN public.employers_industries ei ON e.id = ei.employer_id
GROUP BY e.id
HAVING COUNT(ei.industry_id) = 4
ORDER BY e.name
OFFSET 2 LIMIT 1 
'''

In [65]:
df_5_5 = pd.read_sql_query(query_5_5, connection)
#df_5_5
print('Компания находящаяся на 3 месте по алфавиту, \
у которых заполнено 4 сферы деятельности:', df_5_5.iloc [0][0])

Компания находящаяся на 3 месте по алфавиту, у которых заполнено 4 сферы деятельности: 2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [66]:
query_5_6 = f'''
SELECT COUNT(e.employer_id)
FROM public.employers_industries e
JOIN public.industries i
ON e.industry_id = i.id
WHERE(i.name) LIKE '%Разработка программного обеспечения%' 
'''

In [67]:
df_5_6 = pd.read_sql_query(query_5_6, connection)
#df_5_6
print('Разработка програмного обеспечения указана у', df_5_6.iloc [0][0], 'компаний.')

Разработка програмного обеспечения указана у 3553 компаний.


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [68]:
# код для получения списка городов-милионников
# адрес страницы c городами
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%' +\
      'D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%' +\
      'D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'

# сохранение списка городов вне локальной переменной
cities = []

# внимание: структура сайта может измениться
# в момент выполнения задания нужный список городов,
# находится во второй таблице
with requests.get(url) as response:
    soup = BeautifulSoup(response.text, 'html.parser')
    tables = soup.find_all('tbody')
    rows = tables[1].find_all('tr')
    
    for row in rows[1:]:
        cities.append(row.find('a')['title'])
t_cities= tuple(cities) #превращаем список в кортеж для передачи в запрос
print(t_cities)

('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Челябинск', 'Красноярск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Омск', 'Краснодар', 'Воронеж', 'Пермь', 'Волгоград')


In [69]:
query_5_7 = f'''
(SELECT 
      a.name as "'Регион миллионник'", 
      COUNT(v.id) as "'Колличество вакансий'"
FROM public.areas a
JOIN public.vacancies v ON a.id = v.area_id
JOIN public.employers e ON v.employer_id = e.id
WHERE e.name = 'Яндекс' And a.name in {t_cities}
GROUP BY a.name)

UNION ALL

select 'Total', count(*)
from public.vacancies v
 	join public.employers e on e.id = v.employer_id
 	join public.areas a on a.id = v.area_id
where e.name = 'Яндекс'
  	and a.name in {t_cities} 
order by 2'''

In [70]:
df_5_7 = pd.read_sql_query(query_5_7, connection)
df_5_7

,'Регион миллионник','Колличество вакансий'
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

                                    ВЫВОД
Среди компаний, находящихся в исследуемой базе данных, наибольшее количество вакансий размещают IT компании, банки и компании занимающиеся добычей полезных ископаемых.

Видно, что не все работадатели базы разместили какие-либо вакансиии.

Часть крупных компаний стремятся разместить вакансии во многих регионах. Это связано как с предоставлением услуг в этих регионах, так и вероятно с желанием охватить более широкий круг людей, что увеличивает вероятность получения лучших специалистов в штат.

Чуть меньше, чем треть работадателей не указали сферу деятельности. Возможно в списке сфер деятельности не присутствует необходимых сфер или компания является широко известной в регионе размещения.

15% компаний занимаются разработкой ПО. Можно предположить, что в ближайшее время, количество таких компаний будет расти: постепенно все сферы деятельности человека автоматизируются и роботизируются, и для управления требуются программы.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [71]:
query_6_1 = f'''
SELECT 
      COUNT(v.name)
FROM public.vacancies v
WHERE lower(v.name) LIKE '%data%'
    OR lower(v.name) LIKE '%данн%' 
'''

In [72]:
df_6_1 = pd.read_sql_query(query_6_1, connection)
#df_6_1
print('Отношение к данным имеет', df_6_1.iloc [0][0], 'вакансий.')

Отношение к данным имеет 1771 вакансий.


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [73]:
query_6_2 = f'''
SELECT 
      COUNT(v.name)
FROM public.vacancies v
WHERE ((v.name ILIKE ANY(ARRAY['%data scientist%',
                               '%data science%',
                               '%исследователь данных%',
                               '%machine learning%',
                               '%машинн%обучен%'])
    OR v.name LIKE '%ML%') 
    AND v.name NOT ILIKE '%HTML%')
     
    AND (v.name ILIKE '%junior%' 
    OR v.experience ILIKE '%нет опыта%'
    OR v.employment ILIKE '%стажировка%') 
'''

In [74]:
df_6_2 = pd.read_sql_query(query_6_2, connection)
print('Количество вакансий для начинающего дата-сайентиста составляет:', df_6_2.iloc[0, 0])

Количество вакансий для начинающего дата-сайентиста составляет: 51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [75]:
query_6_3 = f'''
SELECT 
      COUNT(v.name)
FROM public.vacancies v
WHERE ((v.name ILIKE ANY(ARRAY['%data scientist%',
                               '%data science%',
                               '%исследователь данных%',
                               '%machine learning%',
                               '%машинн%обучен%'])
    OR v.name LIKE '%ML%') 
    AND v.name NOT ILIKE '%HTML%')
     
    AND (v.key_skills ILIKE ANY(ARRAY['%sql%', '%postgres%'])) 
'''

In [76]:
df_6_3 = pd.read_sql_query(query_6_3, connection)
#df_6_3
print('Колличество вакансий для DS с навыком SQL или postgres:', df_6_3.iloc [0][0])

Колличество вакансий для DS с навыком SQL или postgres: 201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [77]:
query_6_4 = f'''
SELECT 
      COUNT(v.name)
FROM public.vacancies v
WHERE ((v.name ILIKE ANY(ARRAY['%data scientist%',
                               '%data science%',
                               '%исследователь данных%',
                               '%machine learning%',
                               '%машинн%обучен%'])
    OR v.name LIKE '%ML%') 
    AND v.name NOT ILIKE '%HTML%')
     
    AND (v.key_skills ILIKE '%python%') 
'''

In [78]:
df_6_4 = pd.read_sql_query(query_6_4, connection)
#df_6_4
print('Колличество вакансий для DS с навыком Python:', df_6_4.iloc [0][0])

Колличество вакансий для DS с навыком Python: 351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [79]:
query_6_5 = f'''
SELECT 
      ROUND(avg(cardinality(regexp_split_to_array(key_skills, '\t'))), 2)
FROM public.vacancies v
WHERE ((v.name ILIKE ANY(ARRAY['%data scientist%',
                               '%data science%',
                               '%исследователь данных%',
                               '%machine learning%',
                               '%машинн%обучен%'])
    OR v.name LIKE '%ML%') 
    AND v.name NOT ILIKE '%HTML%') 
'''

In [80]:
df_6_5 = pd.read_sql_query(query_6_5, connection)
#df_6_5
print('В среднем для DS указывают', df_6_5.iloc [0][0],'ключевых навыков.')

В среднем для DS указывают 6.41 ключевых навыков.


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [81]:
query_6_6 = f'''
SELECT
       DISTINCT v.experience as "'Опыт работы'",
       round(avg(coalesce((salary_from+salary_to)/2, salary_from, salary_to))) as "'Средняя зарплата'"
FROM public.vacancies v
WHERE  (LOWER(v.name) LIKE '%data scientist%' OR
       LOWER(v.name) LIKE '%data science%' OR
       LOWER(v.name) LIKE '%исследователь данных%' OR
       (v.name LIKE '%ML%' and v.name NOT ILIKE '%HTML%') OR
       LOWER(v.name) LIKE '%machine learning%' OR
       LOWER(v.name) LIKE '%машинн%обучен%') AND
       (v.salary_from IS NOT NULL or v.salary_to IS NOT NULL) AND
       v.experience IS NOT NULL
GROUP BY v.experience 
'''

In [82]:
df_6_6 = pd.read_sql_query(query_6_6, connection)
display(df_6_6)
print('В среднем дата-сайентист с опытом работы от 3 до 6 лет может рассчитывать\
на зарплату:', df_6_6.iloc [2][1])

,'Опыт работы','Средняя зарплата'
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


В среднем дата-сайентист с опытом работы от 3 до 6 лет может рассчитыватьна зарплату: 243115.0


***

**Выводы по предметному анализу:**

1. 3.5% связаны с профессией дата-сайентиста. Так как количество используемой информации в мире увеличивается, количество вакансий в этой сфере будет расти.
2. Несмотря на 1771 вакансию, которые относится к DS, лишь 51 вакансия  может быть предложена соискателям без опыта работы.
Это 3% всех вакансий в этой сфере. Таким образом, направления деятельности связанные с обучением профессии дата-сайентиста являются выгодными для создания. Так же эта цифра может сказать о том, что рост зарплаты с увелечением опыта работы будет существенным, что было показано в одном из пунктов выше.
3. Среди ключевых навыков Python указывают лишь в 20% случаев, а требования к работе с языком SQL или СУБД Postgres в 11%.
На данный момент анализа прочих требований и ключевых навыков не проводилось, но можно предугадать что в требованиях, указывают более специфичные вещи. Тогда как эти навыки являются основными и их владение подразумевеются без упоминания.
4. Величина зарплаты дата-сайентиста с опытом работы  более трёх лет близится к 250000 рублей, что более чем в два раза превосходит среднее верхнее значение зарплатной вилки из всех профессий безотносительно опыта.

# Дополнительное исследование данных.

Посмотрим на самые распространенные вакансии для DS, Выведим топ 10 по колличеству размещенных вакансий.

In [83]:
query_7_1 = f'''
SELECT 
      v.name as "'Вакансия'",
      COUNT(v.id) as "'Колличество'"
FROM public.vacancies v
WHERE ((v.name ILIKE ANY(ARRAY['%data scientist%',
                               '%data science%',
                               '%исследователь данных%',
                               '%machine learning%',
                               '%машинн%обучен%'])
    OR v.name LIKE '%ML%') 
    AND v.name NOT ILIKE '%HTML%') 
group by 1
order by 2 desc
limit 10 
'''

In [84]:
df_7_1 = pd.read_sql_query(query_7_1, connection)
df_7_1

,'Вакансия','Колличество'
0,Data Scientist,55
1,Senior Data Scientist,21
2,C++ разработчик (QML),10
3,ML Engineer/ Data Scientist (WAF),9
4,Junior Data Scientist (Валидация скоринговых м...,9
5,Middle Data Scientist,8
6,Data scientist,8
7,Архитектор решений с опытом в Tiny ML,6
8,Machine Learning Engineer,6
9,Python-разработчик в команду Data Science,5


                                     Выод
Как видим чаще всего в названии вакансии указывают Data Scientist либо Senior Data Scientist, так же мы видим что присутствует вакансия C++ разработчик (QML) которая врядли относится к DS, что свидетельствует о том что заказчик дал нам не совсем корректное ТЗ в части того какие именно вакансии мы считаем подходящими для DC, что бы продолжить исследования требуется уточнение от заказчика, для подтверждения или корректировки ранее проведенных иследований и продолжения работы над базой данных!

In [85]:
connection.close()

# Общий вывод по проекту

В данном проекте были составлены многочисленных SQL-запросы позволившие изучить рынок вакансий. 
Проект хорошо показывает, возможности по извлечению нужной информации хранящейся в базе данных, применяя предложение SELECT
языка запросов SQL.<br/>

В целом, если говорить про рынок вакансий, то видно что наиболее высокая зарплата у дата-сейнтистов но войти в данную отрасль без опыта достаточно сложно поскольку для начинающих специалистов очень мало предложений, а для специалистов с опытом наоборот, и скорее всего там ощющается кадровый голод. 

Так же мы видим что предложения в основном сконцентрированы в самых больших городах что негативно сказывается на регионах где колличество вакансий вразы меньше а возможность удаленной работы не всегда предусмотрена что в свою очеред вызывает отток специалистов из маленьких регионов и его обеднение.

Так же в ТЗ Проекта обнаружилась не точность в виду чеего вакансия специалиста C++  попала в категорию DS в связи с чем требуется уточнение от заказчика для продолжения исследования и корректировки проведенного. 